# Final Project

**name: Muhammad Albayati**

**SID: 84652863**

In [137]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import meshutils
from camutils import Camera, makerotation, calibratePose, decode, triangulate
import cv2
import visutils
import selectpoints
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial
%matplotlib notebook

In [138]:
def reconstruct(prefix, threshold,camL,camR):
    """
    Simple reconstruction based on triangulating matched pairs of points
    between to view which have been encoded with a 20bit gray code.

    Parameters
    ----------
    prefix : str
      prefix for where the images are stored

    threshold : float
      decodability threshold

    camL,camR : Camera
      camera parameters
      
    posePrefixL : str
      prefix for which pose to load
      
    Returns
    -------
    pts2L,pts2R : 2D numpy.array (dtype=float)

    pts3 : 2D numpy.array (dtype=float)

    """

    imprefixR = prefix + "/frame_C0_"
    imprefixL = prefix + "/frame_C1_"
    
    CLh,maskLh = decode(imprefixL,0,threshold)
    CLv,maskLv = decode(imprefixL,20,threshold)
    CRh,maskRh = decode(imprefixR,0,threshold)
    CRv,maskRv = decode(imprefixR,20,threshold)
    
    leftBgImage = plt.imread(prefix+ "/color_C1_00.png")
    rightBgImage = plt.imread(prefix + "/color_C0_00.png")
    leftFgImage = plt.imread(prefix + "/color_C1_01.png")
    rightFgImage = plt.imread(prefix + "/color_C0_01.png")
    
    if(leftBgImage.dtype == np.uint8):
        leftBgImageGray = leftBgImage.astype(float) / 256
        leftFgImageGray = leftFgImage.astype(float) / 256
        rightBgImageGray = rightBgImage.astype(float) / 256
        rightFgImageGray = rightFgImage.astype(float) / 256
        
    if(len(leftBgImage) > 2):
        leftBgImageGray = np.mean(leftBgImage,axis = 2)
        leftFgImageGray = np.mean(leftFgImage,axis = 2)
        rightBgImageGray = np.mean(rightBgImage,axis = 2)
        rightFgImageGray = np.mean(rightFgImage,axis = 2)

    
    obMaskL = np.abs(leftFgImageGray-leftBgImageGray)>threshold
    obMaskR = np.abs(leftFgImageGray-leftBgImageGray)>threshold
    

    CL = CLh + 1024*CLv
    maskL = maskLh*maskLv*obMaskL
    CR = CRh + 1024*CRv
    maskR = maskRh*maskRv*obMaskR

    h = CR.shape[0]
    w = CR.shape[1]

    subR = np.nonzero(maskR.flatten())
    subL = np.nonzero(maskL.flatten())

    CRgood = CR.flatten()[subR]
    CLgood = CL.flatten()[subL]

    _,submatchR,submatchL = np.intersect1d(CRgood,CLgood,return_indices=True)

    matchR = subR[0][submatchR]
    matchL = subL[0][submatchL]

    xx,yy = np.meshgrid(range(w),range(h))
    xx = np.reshape(xx,(-1,1))
    yy = np.reshape(yy,(-1,1))

    pts2R = np.concatenate((xx[matchR].T,yy[matchR].T),axis=0)
    pts2L = np.concatenate((xx[matchL].T,yy[matchL].T),axis=0)
    
    pts3 = triangulate(pts2L,camL,pts2R,camR)
    
    color = leftFgImage[pts2L[1,:], pts2L[0,:],:].T

    
    return pts2L,pts2R,pts3,color

In [180]:
def generateMesh(path, maskThreshold, camL, camR, pickleFile, blim):
    
    resultFile = pickleFile
    pts2L, pts2R, pts3, color = reconstruct(path, maskThreshold, camL, camR)
    
    
    #
    # drop points which are outside the bounding box
    
    goodpts = np.nonzero((pts3[0,:]>blim[0])&(pts3[0,:]<blim[1]) & \
    (pts3[1,:]>blim[2])&(pts3[1,:]<blim[3])& \
    (pts3[2,:]>blim[4])&(pts3[2,:]<blim[5])) 

    pts3 = pts3[:,goodpts[0]]
    pts2L = pts2L[:,goodpts[0]]
    pts2R = pts2R[:,goodpts[0]]
    color = color[:,goodpts[0]]

    #
    # compute initial triangulation
    
    trithresh = 1
    
    
    Triangles = scipy.spatial.Delaunay (pts2L.T)
    tri = Triangles.simplices

    d01 = np.sqrt(np.sum(np.power(pts3[:,tri[:,0]]-pts3[:,tri[:,1]],2),axis=0))
    d02 = np.sqrt(np.sum(np.power(pts3[:,tri[:,0]]-pts3[:,tri[:,2]],2),axis=0))
    d12 = np.sqrt(np.sum(np.power(pts3[:,tri[:,1]]-pts3[:,tri[:,2]],2),axis=0))

    goodtri = (d01<trithresh)&(d02<trithresh)&(d12<trithresh)
    
    tri = tri[goodtri,:]
    
    toKeep = np.unique(tri)
    indicesMap = np.zeros(pts3.shape[1])
    indicesMap[toKeep] = np.arange(0,toKeep.shape[0])
    indicesMap = indicesMap.astype(int)
    tri = indicesMap[tri]

    pts3 = pts3[:,toKeep]
    pts2L = pts2L[:,toKeep]
    pts2R = pts2R[:,toKeep]
    color = color[:,toKeep]
    
    
    
    
    #334
    
    
    
#     for x in tri:
#         distance1 = np.abs(pts3[:,x[0]] - pts3[:,x[1]])
#         distance2 = np.abs(pts3[:x[0]] - pts3[:,x[2]])
#         average = (distance1 + distance2)/2
#         pts3[:,x[0]] = average
#         distance1 = np.abs(pts3[:,x[1]] - pts3[:,x[0]])
#         distance2 = np.abs(pts3[:x[1]] - pts3[:,x[2]])
#         average = (distance1 + distance2)/2
#         pts3[:,x[1]] = average
#         distance1 = np.abs(pts3[:,x[2]] - pts3[:,x[0]])
#         distance2 = np.abs(pts3[:x[2]] - pts3[:,x[1]])
#         average = (distance1 + distance2)/2
#         pts3[:,x[0]] = average
        
        
    camAngle = path[14:15]
    
    data = {}
    data["pts2L" + str(camAngle)] = pts2L
    data["pts2R" + str(camAngle)] = pts2R
    data["pts3" + str(camAngle)] = pts3
    data["triangles" + str(camAngle)] = tri
    
    fid = open(pickleFile, "wb" ) 
    pickle.dump(data,fid)
    fid.close()
    
    meshutils.writeply(pts3,color,tri,"./meshLab/mesh"+str(camAngle)+".ply")
    
    
#     fig = plt.figure()
#     ax = fig.add_subplot(2,2,1, xlim = (-10, 25), ylim = (0, 30))
#     ax.plot(pts3[0,:],pts3[2,:],'.')
#     plt.title('XZ-view')
#     plt.grid()
#     plt.xlabel('x')
#     plt.ylabel('z')

#     ax = fig.add_subplot(2,2,2, xlim = (0, 30), ylim = (0, 30))
#     ax.plot(pts3[1,:],pts3[2,:],'.')
#     plt.title('YZ-view')
#     plt.grid()
#     plt.xlabel('y')
#     plt.ylabel('z')
    
#     ax = fig.add_subplot(2,2,4, xlim = (-10, 25), ylim = (0, 30))
#     ax.plot(pts3[0,:],pts3[1,:],'.')
#     plt.title('XY-view')
#     plt.grid()
#     plt.xlabel('x')
#     plt.ylabel('y')
    

In [181]:
camParams = np.load("calibration.pickle")
focalLength = (camParams["fx"] + camParams["fy"])/2
prinPoint = np.array([[camParams["cx"]], 
                     [camParams["cy"]]])

rotationMatrix = np.array([[0, 0, 0],
                          [0, 0, 0], 
                          [0, 0, 0]])

translationMatrix = np.array([[0],
                            [0],
                            [0]])

camL = Camera(focalLength, prinPoint, rotationMatrix, translationMatrix)
camR = Camera(focalLength, prinPoint, rotationMatrix, translationMatrix)


imgL = plt.imread('./calib_jpg_u/frame_C1_01.jpg')
ret, cornersL = cv2.findChessboardCorners(imgL, (8,6), None)
pts2L = cornersL.squeeze().T

imgR = plt.imread('./calib_jpg_u/frame_C0_01.jpg')
ret, cornersR = cv2.findChessboardCorners(imgR, (8,6), None)
pts2R = cornersR.squeeze().T

pts3 = np.zeros((3,6*8))
yy,xx = np.meshgrid(np.arange(8),np.arange(6))
pts3[0,:] = 2.8*xx.reshape(1,-1)
pts3[1,:] = 2.8*yy.reshape(1,-1)

params_init = np.array([0, 0, 0, 0, 0, -2])

camL = calibratePose(pts3, pts2L, camL, params_init)
camR = calibratePose(pts3, pts2R, camR, params_init)

In [182]:
pickleFile = "./mesh.pickle"
boxLimits = [-100, 100, -100, 100, -100, 100]
generateMesh("./teapot/grab_0_u", 0.02, camL, camR, pickleFile, boxLimits)
generateMesh("./teapot/grab_1_u", 0.02, camL, camR, pickleFile, boxLimits)
generateMesh("./teapot/grab_2_u", 0.02, camL, camR, pickleFile, boxLimits)
# boxLimits = [-1, 19, 6, 17, 14, 24]
generateMesh("./teapot/grab_3_u", 0.02, camL, camR, pickleFile, boxLimits)
# boxLimits = [-5, 21, 3, 25, 14, 24]
generateMesh("./teapot/grab_4_u", 0.02, camL, camR, pickleFile, boxLimits)
generateMesh("./teapot/grab_5_u", 0.02, camL, camR, pickleFile, boxLimits)
generateMesh("./teapot/grab_6_u", 0.02, camL, camR, pickleFile, boxLimits)

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )

In [80]:
x = 0
while(x < 7):
    x += 1
    
path = "./teapot/grab_" + str(x) + "_u"
angle = path[14:15]

7
